In [18]:
import pandas as pd
import plotly.express as px


In [19]:
df = pd.read_parquet('/home/bianka/PISI3_2022.2/data/dataset_renomeado.parquet')
df = df[['ventilado_apache','tipo_estadia_uti', 'd1_frequencia_cardiaca_maxima',
         'd1_frequencia_cardiaca_minima', 'h1_frequencia_respiratoria_maxima',
         'h1_frequencia_respiratoria_minima','h1_spO2_maximo',
         'd1_spO2_minimo','d1_temperatura_minima',
         'd1_temperatura_maxima', 'morte_hospital']]
df = df.dropna()


Verificando quantas colunas ficaram após remover todas as linhas vazias das colunas selecionadas

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83455 entries, 0 to 91712
Data columns (total 11 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   ventilado_apache                   83455 non-null  float64
 1   tipo_estadia_uti                   83455 non-null  object 
 2   d1_frequencia_cardiaca_maxima      83455 non-null  float64
 3   d1_frequencia_cardiaca_minima      83455 non-null  float64
 4   h1_frequencia_respiratoria_maxima  83455 non-null  float64
 5   h1_frequencia_respiratoria_minima  83455 non-null  float64
 6   h1_spO2_maximo                     83455 non-null  float64
 7   d1_spO2_minimo                     83455 non-null  float64
 8   d1_temperatura_minima              83455 non-null  float64
 9   d1_temperatura_maxima              83455 non-null  float64
 10  morte_hospital                     83455 non-null  int64  
dtypes: float64(9), int64(1), object(1)
memory usage: 7.6+ 

Transformando a coluna categórica do de tipo_estadia_uti em colunas binárias 

In [21]:

one_hot_encoded = pd.get_dummies(df['tipo_estadia_uti'])

df = pd.concat([df, one_hot_encoded], axis=1)


## Rotulação das colunas e conversão para binário para tentar melhorar os resultados do KNN

### Coluna primeiras_24_horas_frequencia_cardiaca_maxima

#### Categorização da coluna d1_frequencia_cardiaca_maxima

In [22]:
def categorizar_fc_maxima(fc):
    if fc < 60:
        return 'primeiras_24_horas_frequencia_cardiaca_maxima_muito_baixa'
    elif 60 <= fc < 70:
        return 'primeiras_24_horas_frequencia_cardiaca_maxima_baixa'
    elif 70 <= fc < 100:
        return 'primeiras_24_horas_frequencia_cardiaca_maxima_normal'
    elif 100 <= fc < 120:
        return 'primeiras_24_horas_frequencia_cardiaca_maxima_elevada'
    else:
        return 'primeiras_24_horas_frequencia_cardiaca_maxima_muito_elevada'

df['categoria_fc_maxima'] = df['d1_frequencia_cardiaca_maxima'].apply(categorizar_fc_maxima)

one_hot_encoded = pd.get_dummies(df['categoria_fc_maxima'])

df = pd.concat([df, one_hot_encoded], axis=1)

df = df.drop('categoria_fc_maxima', axis=1)



### Coluna primeiras_24_horas_frequencia_cardiaca_minima

remoção de todas as linhas com a frequência cardiaca menor que 10

In [23]:
df = df[~((df['d1_frequencia_cardiaca_minima'] >= 0) & (df['d1_frequencia_cardiaca_minima'] <= 9))]


In [24]:
def categorizar_fc_minima(fc):
    if fc < 60:
        return 'primeiras_24_horas_frequencia_cardiaca_minima_muito_baixa'
    elif 60 <= fc < 70:
        return 'primeiras_24_horas_frequencia_cardiaca_minima_baixa'
    elif 70 <= fc < 100:
        return 'primeiras_24_horas_frequencia_cardiaca_minima_normal'
    elif 100 <= fc < 120:
        return 'primeiras_24_horas_frequencia_cardiaca_minima_elevada'
    else:
        return 'primeiras_24_horas_frequencia_cardiaca_minima_muito_elevada'

df['categoria_fc_minima'] = df['d1_frequencia_cardiaca_minima'].apply(categorizar_fc_maxima)

df = df.drop('categoria_fc_minima', axis=1)



### Colunas h1_frequencia_respiratoria_maxima

In [25]:
def categorizar_fr(fr):
    if fr < 12:
        return 'primeira_hora_frequencia_respiratoria_maxima_muito_baixa'
    elif 12 <= fr < 16:
        return 'primeira_hora_frequencia_respiratoria_maxima_baixa'
    elif 16 <= fr < 20:
        return 'primeira_hora_frequencia_respiratoria_maxima_normal'
    elif 20 <= fr < 24:
        return 'primeira_hora_frequencia_respiratoria_maxima_elevada'
    else:
        return 'primeira_hora_frequencia_respiratoria_maxima_muito elevada'

df['categoria_frequencia_respiratoria_maxima'] = df['h1_frequencia_respiratoria_maxima'].apply(categorizar_fr)

one_hot_encoded = pd.get_dummies(df['categoria_frequencia_respiratoria_maxima'])

df = pd.concat([df, one_hot_encoded], axis=1)

df = df.drop('categoria_frequencia_respiratoria_maxima', axis=1)


### Colunas d1_spO2_minimo

Saturação 


Remoção de todas as linhas com a saturação menor que 20

In [26]:
df = df[~((df['d1_spO2_minimo'] <= 20))]


In [27]:
def categorize_saturacao(sat):
    if sat < 85:
        return 'saturacao_primieras_vinte_quatro_horas_minima_muito_baixa'
    elif sat < 90:
        return 'saturacao_primieras_vinte_quatro_horas_minima_baixa'
    elif sat < 95:
        return 'saturacao_primieras_vinte_quatro_horas_minima_moderada'
    elif sat < 100:
        return 'saturacao_primieras_vinte_quatro_horas_minima_boa'


df['categoria_d1_spO2_minimo'] = df['d1_spO2_minimo'].apply(categorizar_fr)

one_hot_encoded = pd.get_dummies(df['categoria_d1_spO2_minimo'])

df = pd.concat([df, one_hot_encoded], axis=1)

df = df.drop('categoria_d1_spO2_minimo', axis=1)


### Colunas d1_temperatura_minima


In [28]:
def categorize_temperatura(temp):
    if temp < 35:
        return 'temperatura_minima_primieras_vinte_quatro_horas_hipotermia'
    if temp < 37:
        return 'temperatura_minima_primieras_vinte_quatro_horas_normal'
    if temp >= 37:
        return 'temperatura_minima_primieras_vinte_quatro_horas_febre'

df['categoria_d1_temperatura_minima'] = df['d1_temperatura_minima'].apply(categorize_temperatura)

one_hot_encoded = pd.get_dummies(df['categoria_d1_temperatura_minima'])

df = pd.concat([df, one_hot_encoded], axis=1)

df = df.drop('categoria_d1_temperatura_minima', axis=1)


In [29]:
df = df.drop(['d1_frequencia_cardiaca_maxima',
       'd1_frequencia_cardiaca_minima', 'h1_frequencia_respiratoria_maxima',
       'd1_spO2_minimo', 'd1_temperatura_minima','tipo_estadia_uti'], axis=1)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82600 entries, 0 to 91712
Data columns (total 23 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   ventilado_apache                                             82600 non-null  float64
 1   h1_frequencia_respiratoria_minima                            82600 non-null  float64
 2   h1_spO2_maximo                                               82600 non-null  float64
 3   d1_temperatura_maxima                                        82600 non-null  float64
 4   morte_hospital                                               82600 non-null  int64  
 5   admit                                                        82600 non-null  uint8  
 6   readmit                                                      82600 non-null  uint8  
 7   transfer                                                     82600 non-null 

In [31]:
df = df.astype(int)


In [32]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from imblearn.over_sampling import SMOTE


y = df['morte_hospital']
X = df.drop(['morte_hospital'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

knn_classifier = KNeighborsClassifier(n_neighbors=3)  

knn_classifier.fit(X_train_scaled, y_train)


y_pred = knn_classifier.predict(X_test_scaled)

# Avaliar o desempenho do modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo: {accuracy:.2f}")

print("Relatório de classificação:")
print(classification_report(y_test, y_pred))

print("Matriz de confusão:")
print(confusion_matrix(y_test, y_pred))


Acurácia do modelo: 0.90
Relatório de classificação:
              precision    recall  f1-score   support

           0       0.93      0.97      0.95     15139
           1       0.32      0.14      0.19      1381

    accuracy                           0.90     16520
   macro avg       0.62      0.56      0.57     16520
weighted avg       0.87      0.90      0.89     16520

Matriz de confusão:
[[14724   415]
 [ 1188   193]]


## Aplica o Random Under Sampling (RUS) 

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from imblearn.under_sampling import RandomUnderSampler

y = df['morte_hospital']
X = df.drop(['morte_hospital'], axis=1)

X_train_temp, X_test, y_train_temp, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)

# Aplicar undersampling apenas nos dados de treinamento
rus = RandomUnderSampler(sampling_strategy='auto', random_state=42)  
X_train_resampled, y_train_resampled = rus.fit_resample(X_train_temp, y_train_temp)

X_train, _, y_train, _ = train_test_split(X_train_resampled, y_train_resampled, test_size=0.33, random_state=42, stratify=y_train_resampled)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

knn_classifier = KNeighborsClassifier(n_neighbors=4)  
knn_classifier.fit(X_train_scaled, y_train)

y_pred = knn_classifier.predict(X_test_scaled)

# Avaliar o desempenho do modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo: {accuracy:.2f}")

print("Relatório de classificação:")
print(classification_report(y_test, y_pred))

print("Matriz de confusão:")
print(confusion_matrix(y_test, y_pred))


Acurácia do modelo: 0.75
Relatório de classificação:
              precision    recall  f1-score   support

           0       0.95      0.77      0.85     25066
           1       0.17      0.53      0.26      2192

    accuracy                           0.75     27258
   macro avg       0.56      0.65      0.55     27258
weighted avg       0.89      0.75      0.80     27258

Matriz de confusão:
[[19391  5675]
 [ 1030  1162]]
